In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, _tr, Timestamp, Context
from mand.core import ProfileMonitor, PrintMonitor
from mand.lib.extrefdata import ExternalRefData, dataField
from mand.lib.workflow import Workbook, WorkItemOpenEvent, WorkItem
from mand.lib.portfolio import Portfolio
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import num, find
import datetime
from mand.lib.dbsetup import setUpDb

from mand.demos.trading import TradingBook, TradingPortfolio, MarketDataSource, MarketInterface

db = ObjectDb()
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x106220ad0>: 107, mem=True, ro=False: entities=9, map=2


In [2]:
def makeTree(names):
    ret = []
    for name in names:
        subs = [ TradingBook(name+str(i)) for i in range(10) ]
        p = TradingPortfolio(name).write()
        p.setChildren(subs)
        ret.append(p)
    return ret

with db:
    pAll = TradingPortfolio('TopOfTheHouse').write()
    subs = makeTree(['Eq-Prop', 'Eq-Inst', ])# 'FX', 'Rates', 'Credit', 'Delta1', 'Loans', 'Commod', 'ETFs', 'Mtge'])
    pAll.setChildren(subs)
    
print pAll
print '# books:', len(pAll.books())
print '# children:', len(pAll.children())

# books: 20
# children: 2


In [3]:
with db:
    bExt  = _tr.TradingBook('Customer1')
    bExt2 = _tr.TradingBook('Customer2')
    
p1 = pAll.children()[0]
p2 = pAll.children()[1]

b1 = p1.children()[0]
b2 = p2.children()[0]

print bExt.meta.name()
print b1.meta.name()
print b2.meta.name()

Customer1
Eq-Prop0
Eq-Inst0


In [4]:
with db:
    s1_ibm  = MarketDataSource('source1.IBM')
    s1_goog = MarketDataSource('source1.GOOG')

s1_ibm.update(last=175.61)
s1_goog.update(last=852.12)

In [5]:
with db:
    ibm  = MarketInterface('IBM')
    goog = MarketInterface('GOOG')

In [6]:
with db:
    TradeOpenEvent = _tr.TradeOpenEvent
    cf1 = _tr.ForwardCashflow()
    ins1 = _tr.Equity()
    ins2 = _tr.Equity(assetName='GOOG.Eq.1')
    
    ts1 = Timestamp()
    
    ev1 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.65,
                         book1=b1,
                         book2=bExt).write()
    
    ts2 = Timestamp()
    
    s1_ibm.update(last=175.64)
    
    ts3 = Timestamp()
    
    ev2 = TradeOpenEvent(action='Buy',
                         item=ins2,
                         quantity=300,
                         premium=cf1,
                         unitPrice=852.12,
                         book1=b2,
                         book2=bExt).write()
    
    ev3 = TradeOpenEvent(action='Sell',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.85,
                         book1=b2,
                         book2=bExt2).write()
    
    ts4 = Timestamp()
    
    s1_ibm.update(last=175.70)
    s1_goog.update(last=852.11)
    
    ts5 = Timestamp()
    
    s1_ibm.update(last=175.68)
    s1_goog.update(last=852.13)
    
    eod = Timestamp()
    
    ev4 = TradeOpenEvent(action='Buy',
                         item=ins1,
                         quantity=100,
                         premium=cf1,
                         unitPrice=175.69,
                         book1=b1,
                         book2=bExt,
                         amends=ev1,
                         message='Sorry, the broker says you actually paid 69. signed: the middle office'
                        ).write(validTime=ev1.meta._timestamp.validTime)
    
    s1_ibm.update(last=177.68)
    s1_goog.update(last=856.13)
    
    ts6 = Timestamp()
    

In [7]:
pm = None

class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        def clocks(ts):
            def fn(node):
                obj = node.key[0]
                m = node.key[1].split(':')[-1]
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return dict( [ (node.tweakPoint, node) for node in nodes ] )
    
        allNodes = clocks(ts1)
    
        allNodes.update(clocks(ts2))
        nodes = allNodes.values() 
    
        # IRL, we'd sort these according to some business req...
        nodes = sorted(nodes, key = lambda node: node.key[0].meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n.tweakPoint] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        for n in nodes:
            tweaks[n.tweakPoint] = tsAmend
            name = n.key[0].meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n.tweakPoint] = ts2
            name = n.key[0].meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        displayHeader('%s' % title)
        displayListOfDicts(data, names=['Activity', 'PnL'] )
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 1196.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

In [8]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

In [9]:
db3 = db.copy()
p = db3._get(pAll.meta.path())
    
# Note misleading cum times below due to recursion...

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times
LogMessage: Oops, book appears multiple times


# PnL explain for TopOfTheHouse: 2216.00

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|1200.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Portfolio:items|33|2,532,120|279|76,730|GetValue
|Portfolio:items|33|2,531,841|5,445|76,722|GetValue/Calc
|Report:data|1|1,499,593|10|1,499,593|GetValue
|Report:data|1|1,499,582|361|1,499,582|GetValue/Calc
|TradingContainer:NPV|11|1,494,647|113|135,877|GetValue
|TradingContainer:NPV|11|1,494,534|1,476|135,866|GetValue/Calc
|Workbook:items|224|1,033,570|1,980|4,614|GetValue
|Workbook:items|220|1,031,590|917,180|4,689|GetValue/Calc
|Root:Clocks|2|505,798|797|252,899|Context
|Portfolio:children|66|267,353|430|4,050|GetValue
|Portfolio:children|33|266,922|138,249|8,088|GetValue/Calc
|Equity:NPV|15|184,614|167|12,307|GetValue
|Equity:NPV|15|184,446|381|12,296|GetValue/Calc
|MarketInterface:spot|15|170,474|124|11,364|GetValue
|MarketInterface:spot|15|170,350|459|11,356|GetValue/Calc
|ExternalRefData:state|15|161,431|105|10,762|GetValue
|ExternalRefData:state|15|161,325|283|10,755|GetValue/Calc
|RefData:state|15|161,041|118|10,736|GetValue
|RefData:state|15|160,922|72,446|10,728|GetValue/Calc
|Clock:cutoffs|544|140,428|2,021|258|GetValue
|Clock:cutoffs|20|138,406|408|6,920|GetValue/Calc
|Root:End|1|126,748|35|126,748|Context
|Root:Start|1|121,417|15|121,417|Context
|Root:Activity Trading|1|108,735|34|108,735|Context
|Root:Activity MarketData|1|108,043|35|108,043|Context
|Root:Amend Portfolio|1|107,815|39|107,815|Context
|Root:Activity Portfolio|1|107,685|39|107,685|Context
|Root:Amend MarketData|1|103,942|32|103,942|Context
|Clock:parent|20|103,797|156|5,189|GetValue
|Clock:parent|20|103,640|84,275|5,182|GetValue/Calc
|Root:Start breaks|1|103,267|32|103,267|Context
|Root:Amend Trading|1|102,292|36|102,292|Context
|TradingBook|22|86,039|86,039|3,910|Db.Get
|PortfolioUpdateEvent:children|33|86,027|507|2,606|GetValue
|RefDataUpdateEvent|9|40,228|40,228|4,469|Db.Get
|Clock|5|21,116|21,116|4,223|Db.Get
|PortfolioUpdateEvent|4|20,166|20,166|5,041|Db.Get
|TradeOpenEvent|4|17,888|17,888|4,472|Db.Get
|TradeOpenEvent:ticket|66|16,317|368|247|GetValue
|TradingTicket|4|15,949|15,949|3,987|Db.Get
|Equity:refdata|15|13,590|259|906|GetValue
|Equity:refdata|15|13,331|415|888|GetValue/Calc
|TradingBook:clock|440|12,170|4,141|27|GetValue
|ClockEvent|2|10,151|10,151|5,075|Db.Get
|_WorkItemEvent:book2|66|9,194|334|139|GetValue
|MarketInterface|2|9,055|9,055|4,527|Db.Get
|MarketInterface:source|15|8,459|150|563|GetValue
|ClockEvent:parent|8|8,445|71|1,055|GetValue
|TradingPortfolio|2|8,340|8,340|4,170|Db.Get
|MarketInterface:source|15|8,308|349|553|GetValue/Calc
|TradingBook:clock|220|8,028|3,398|36|GetValue/Calc
|MarketDataSource|2|7,768|7,768|3,884|Db.Get
|_WorkItemEvent:item|33|7,736|271|234|GetValue
|Equity|2|7,464|7,464|3,732|Db.Get
|Portfolio:clock|66|5,473|593|82|GetValue
|Portfolio:clock|33|4,879|483|147|GetValue/Calc
|MarketDataSource:clock|30|4,186|255|139|GetValue
|MarketDataSource:clock|15|3,931|216|262|GetValue/Calc
|MarketInterface|1|3,747|3,747|3,747|Db.Put
|TradeOpenEvent:premium|33|3,665|222|111|GetValue
|RootClock|1|3,458|3,458|3,458|Db.Get
|ForwardCashflow|1|3,443|3,443|3,443|Db.Get
|Portfolio:books|55|1,855|408|33|GetValue
|Portfolio:books|33|1,446|1,213|43|GetValue/Calc
|Event:amends|142|1,049|1,049|7|GetValue
|Entity:clock|40|574|357|14|GetValue
|_WorkItemEvent:book1|66|323|323|4|GetValue
|TradeOpenEvent:quantity|66|293|293|4|GetValue
|RefDataUpdateEvent:data|58|256|256|4|GetValue
|Entity:clock|20|216|216|10|GetValue/Calc
|MarketInterface:sourceName|15|190|129|12|GetValue
|TradeOpenEvent:action|33|183|183|5|GetValue
|RootClock:cutoffs|52|181|181|3|GetValue
|TradeOpenEvent:unitPrice|33|180|180|5|GetValue
|ForwardCashflow:NPV|11|173|118|15|GetValue
|Equity:assetName|15|112|112|7|GetValue
|MarketInterface:sourceName|15|61|61|4|GetValue/Calc
|ForwardCashflow:NPV|11|54|54|4|GetValue/Calc
|Report:valuable|1|5|5|5|GetValue
|Report:ts1|1|5|5|5|GetValue
|Report:ts2|1|3|3|3|GetValue